# Data feed (optional)

Using a notebook to feed data into Event Store is now optional for this code pattern.
Instead, you should use IBM Streams to feed the data. If you are not using IBM Streams,
you can run this notebook as an alternative.

In this notebook, a CSV file is read into a pandas DataFrame
which is then used to batch insert the rows into Event Store.

Before running this notebook, you need to run the **Event Store Table Creation**
notebook to setup the database and table.

## Connect to IBM Db2 Event Store

### Determine the IP address of your host

Obtain the IP address of the host that you want to connect to by running the appropriate command for your operating system:

* On Mac, run: `ifconfig`
* On Windows, run: `ipconfig`
* On Linux, run: `hostname -i`

Edit the `HOST = "XXX.XXX.XXX.XXX"` value in the next cell to provide the IP address.

In [21]:
# Set your host IP address
HOST = "192.168.0.104"

# Port will be 1100 for version 1.1.2 or later (5555 for version 1.1.1)
PORT = "1100"

# Database name
DB_NAME = "TESTDB"

# Table name
TABLE_NAME = "IOT_TEMPERATURE"

## Import Python modules


In [22]:
from eventstore.common import ConfigurationReader
from eventstore.oltp import EventContext

## Connect to Event Store

In [23]:
endpoint = HOST + ":" + PORT
print("Event Store connection endpoint:", endpoint)
ConfigurationReader.setConnectionEndpoints(endpoint)

Event Store connection endpoint: 192.168.0.104:1100


## Load the tables and inspect the table schemas

The following cell gets the schema of the table.

In [24]:
ctx = EventContext.get_event_context(DB_NAME)
resolved_table_schema = ctx.get_table(TABLE_NAME)
print(resolved_table_schema)

ResolvedTableSchema(tableName=IOT_TEMPERATURE, schema=StructType(List(StructField(deviceID,IntegerType,false),StructField(sensorID,IntegerType,false),StructField(ts,LongType,false),StructField(ambient_temp,DoubleType,false),StructField(power,DoubleType,false),StructField(temperature,DoubleType,false))), sharding_columns=['deviceID', 'sensorID'], pk_columns=['deviceID', 'sensorID', 'ts'], partition_columns=None)


## Load the sample data

Let's insert some sample data into the table before starting analysis.
To load data into the table, you will have to:

- Use the Event Store UI to add the "sample_IOT_table.csv" file as a project data asset.
- Run the following cells to load the data from **assets**.

### Read the CSV into a pandas DataFrame

In [25]:
import pandas as pd
df = pd.read_csv("assets/sample_IOT_table.csv")
df

,1,48,1541019342393,25.983183481618322,14.65874116573845,48.908846094198
0,1,24,1541019343497,22.545444,9.834895,39.065559
1,2,39,1541019344356,24.324654,14.100638,44.398837
2,2,1,1541019345216,25.658281,14.243132,45.291255
3,2,20,1541019346515,26.836546,12.841558,48.700130
4,1,24,1541019347200,24.960868,11.773728,42.161830
5,1,35,1541019347966,23.702428,7.518410,40.792013
6,1,32,1541019348864,24.041499,10.201932,41.664663
7,2,47,1541019349485,27.085396,7.805625,45.607395
8,1,12,1541019349819,20.633590,10.344878,37.514075
9,1,4,1541019350783,23.012288,2.447689,34.796619


### Examine the data types

In [26]:
df.dtypes

1                       int64
48                      int64
1541019342393           int64
25.983183481618322    float64
14.65874116573845     float64
48.908846094198       float64
dtype: object

### Batch insert the data from the DataFrame into the table

In [27]:
ctx.batch_insert(resolved_table_schema, df.to_records(index=False).tolist())

## Summary
In this notebook, you learned how to:
- Insert data into a table from a CSV file

<p><font size=-1 color=gray>
&copy; Copyright 2019 IBM Corp. All Rights Reserved.
<p>
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.
</font></p>